In [18]:
'''
zig-zag position in 2d
- S: Space s x s
- b: init position on the cols (based on the start point of the cycle)
- x: current position along the rows

{
    # Here S is also gonna be translated into S_p for position handling

    -------------------------
    Down
    f(x) = [x, b]                 | IF x <= S

    -------------------------
    Diagonal
    This parts requires normalization, since we are working with a new pivot with linear values, and the first drop goes S elements and so the diagonal, we can reset the indexing back to 0 in the arrangement of 0,...,S

    Also since the diagonal always goes from bottom left to top right, it goes up, but since the array we do not count in an inverted manner of the cartesian graph, we should inverse the positions

    • Normalization
    x_n = (x - S)

    • Diagonal Inversion
    x_ni = (S - x_n)

    f(x_n) = [S-x_n, b+x_n]        | IF x > S
}
'''

def zig_zag_2d_positioner(S: int, x: int, b: int) -> list:
    """
    Calculate the 2D position for an element in a zigzag pattern within a square grid.

    Parameters:
    - S (int): Dimension of the square grid (number of rows/columns).
    - x (int): Current position in the zigzag traversal, where x is zero-indexed.
    - b (int): Initial column offset for the start of the cycle.

    The function handles a zigzag pattern where elements first move straight down and then diagonally up-right.

    The path is split into two phases:
    1. Downward: Directly downward until the end of the grid.
    2. Diagonal: Moves diagonally up-right starting from the end of the downward path.

    The function calculates the position by normalizing and inverting the path during the diagonal phase.

    Returns:
    list: A list containing two integers [row_index, col_index] representing the position of 'x' in the grid.

    Raises:
    ValueError: If 'x' is outside the valid range of [0, 2S-2] inclusive.

    Example:
    >>> zig_zag_2d_positioner(5, 7, 0)
    [2, 2]
    """

    # Compute the maximum index in the zigzag cycle
    # max_cycle_index = C
    C = 2 * S - 2
    if x < 0 or x >= C:
        raise ValueError(f'x is out of the allowed cycle range, which is from 0 to {C}')

    # Determine the boundary within the grid where the downward movement ends
    downward_boundary = S - 1

    # Down pattern: movement is strictly downward
    if x <= downward_boundary:
        return [x, b]

    # Inverse diagonal pattern starts after the end of the downward pattern
    # Normalization: shift by the downward boundary to recalculate position for diagonal phase
    normalized_x = x - downward_boundary
    # Diagonal inversion: reflects the diagonal movement to simulate upward movement on the grid
    inverted_x = downward_boundary - normalized_x

    # Compute the final position considering the initial column offset and the normalized diagonal movement
    return [inverted_x, b + normalized_x]


In [12]:
# Test Cases
S = 4
x = 0
b = 0

C = S*2 - 2 # limit of the table cases for x
C

6

In [16]:
'''
Table Cases
• S = 4
• C = S*2 - 2 = 6

x, b    |   y
--------|---------
0, 0    |   (0, 0)
1, 0    |   (1, 0)
2, 0    |   (2, 0)
3, 0    |   (3, 0)
4, 0    |   (2, 1)
5, 0    |   (1, 2)
'''

coordinates = []
coordinates.append(zig_zag_2d_positioner(S, 0, 0))
coordinates.append(zig_zag_2d_positioner(S, 1, 0))
coordinates.append(zig_zag_2d_positioner(S, 2, 0))
coordinates.append(zig_zag_2d_positioner(S, 3, 0))
coordinates.append(zig_zag_2d_positioner(S, 4, 0))
coordinates.append(zig_zag_2d_positioner(S, 5, 0))
coordinates

[[0, 0], [1, 0], [2, 0], [3, 0], [2, 1], [1, 2]]

In [ ]:
# Error case, out of boundaries
zig_zag_2d_positioner(S, 6, 0)  # ValueError: x is out of the allowed cycle range, which is from 0 to 5

---

## Cycle Sequence - Adjacent

##### Normalization
We need to first normalize the X n° of element in order to determinate the `relative position` around the uncovered cycle to work in an `stateless` manner with our previous system of equations and so for the `b` position

• c = X // C        | n° completed cycles
• r = X % C         | n° remanent steps along the incompleted cycle or n° element in the incompleted cycle

**Special Case**
In case X == C, it means a cycle is fully completed but no more remanent steps on an uncovered cycle. Meaning i should consider the last step of completed circle, relatively

if r == 0; r == C (in order to have the last )

now we have the exact number of cycles (c) and where we currently are along the uncovered cycle (r)

**r, x_p and b parameter**
now we need to calculate b to know the starting position of the current cycle. This will also affect r in an special case where there is no remanent.

In case of `r` we can normalize it into the x position called `x_p`
> {
>     x_p = r-1      | IF r > 0
>     x_p = C        | IF r == 0
> }

In case of `b`
B defines where along the space, is the searched position, it depends on the number of cycles (c) and remanent (r) steps. Its a measurement of position in this case and based on the pattern, it will indicate the `axis x` or cols

Full Cycle Cols
> CS = 1 + (S-2)
> {
>   b = CS * (c-1)   | IF !r (no remanent)
>   b = CS * (c)     | IF r (remanent)
> }

After we get c, r, and b we can send b, but x should be along the range or predispose limits of our stateless pattern function, so x is not literal x, but the relative position, that is represented in this case by r, the r that counts x from the beggining of the cycle

we send b, and finally r as x

b = position measure
r as x = position measure

In [23]:
'''
This Section introduces Modular Arithmetic that will help with the cycling around the zig-zag pattern
'''

def zig_zag_2d_positioner_normalization(S: int, x: int) -> list:
    """
    From S Space, we will determinate:
    - C: Cycle limit = 2S - 2
    - CS: represents the number of cols a cycle occupies (helpful to determine the b normalized position) = 1 + (S-2)
    - c: n° of completed cycles
    - r: n° of remaining elements in the cycle

    From this calculated parameters, we can determine the normalized position
    - b: initial column offset of a cycle, always from the beggining of the current cycle
    - x_n: normalized position
    [x_n, b]
    
    This allow us using the cycle function in an stateless manner
    """
    # parameters
    C = 2*S - 2
    cols_per_cycle = 1 + (S-2)
    c = x // C
    r = x % C

    # normalization
    if r == 0:
        x_normalized = C-1                    # last position of current cycle
        b_normalized = cols_per_cycle * (c-1) # init of previous cycle
    else:
        x_normalized = r-1                    # position in the current cycle
        b_normalized = cols_per_cycle * c     # init of current cycle
    
    return [x_normalized, b_normalized]


In [24]:
'''
Test Cases
• S = 4
• C = 2S - 2 = 6
• CS = 1 + (S-2) = 3

x    |   c, r    |   x_n, b
-----|-----------|-----------
7    |   1, 1    |   (0, 3)
8    |   1, 2    |   (1, 3)
9    |   1, 3    |   (2, 3)
10   |   1, 4    |   (3, 3)
11   |   1, 5    |   (4, 3)
12   |   2, 0    |   (5, 3)
'''

S = 4

coordinates = []

for i in range(7, 13):
    coordinates.append(zig_zag_2d_positioner_normalization(S, i))

coordinates

[[0, 3], [1, 3], [2, 3], [3, 3], [4, 3], [5, 3]]

In [ ]:
def zig_zag_2d_positioner_normalization(S: int, x: int) -> list:
    """
    Normalize the position and cycle starting column based on a given index in a zigzag pattern.
    
    Parameters:
    - S (int): Dimension of the square grid (number of rows/columns).
    - x (int): Zero-indexed position in the sequence.

    Returns:
    - list: A list containing the normalized position in the cycle [x_n] and the starting column [b] of that cycle.
    
    This function uses modular arithmetic to calculate the position within the cycle and the cycle's starting column.
    The cycle's length and the number of columns it spans are derived from the grid dimension (S).
    """
    C = 2 * S - 2  # Total number of positions in a full cycle
    cols_per_cycle = max(1, S - 1)  # Calculate columns per cycle, ensure non-zero
    c = x // C  # Number of complete cycles
    r = x % C  # Position within the current cycle

    if r == 0:
        x_normalized = C - 1  # Position at the end of a cycle
        b_normalized = cols_per_cycle * (c - 1)  # Start column for the cycle
    else:
        x_normalized = r - 1  # Zero-indexed position in the cycle
        b_normalized = cols_per_cycle * c  # Start column for the current cycle

    return [x_normalized, b_normalized]

# Test the normalization function
S = 4
test_range = range(7, 13)  # Define a range of test positions
coordinates = [zig_zag_2d_positioner_normalization(S, i) for i in test_range]
print(coordinates)  # Expected output: [[0, 3], [1, 3], [2, 3], [3, 3], [4, 3], [5, 3]]

In [26]:
class ZigZagPositioner:
    """
    ZigZagPositioner class to handle 2D position calculations in a zigzag pattern.
    • S: represents the space in rows x cols             -> S = S x S
    • C: represents the defined cycle based on the space -> C = 2S - 2
    • CS: represents the number of cols a cycle occupies -> CS = 1 + (S-2) (in this case represented as cols, but not position)
          might be useful to translate the concept into a positional term
    • downward_boundary (int): Boundary where the downward movement ends, S - 1.

    -------------------------
    Stateless Cycle
    • b: initial column offset for the start of the cycle          
    • x: current position along the rows

    > Boundaries
    • range: [0, C-1]
    • downward_boundary: represents the boundary where the downward movement ends       = S - 1
    • diagonal_boundary: represents the boundary where the diagonal movement starts     = > S

    -------------------------
    Sequential Cycle Repetition: 
    the cycle is repeated adjacent to the previous cycle, we can have a continous 'x' position, but we need to normalize it to a cycle

    > Normalization
    • c: represents the number of completed cycles
    • r: represents the number of remaining elements in the cycle

    - normalized position
    • x_n: normalized position
    • b: initial column offset for the start of the cycle

        2 edge main cases to define the normalized position (x and b):
        \_ Completed Cycle:      begins from the last completed cycle
        \_ Uncompleted Cycle:    begins from the start of the current uncompleted cycle
    """

    def __init__(self, S:int):
        """
        Initialize the ZigZagPositioner with the specified grid size.
        
        Parameters:
            S (int): The size of the grid (number of rows and columns).
        """
        # validation
        if S < 2:
            raise ValueError('The space dimension must be at least 2')
        
        # Space
        self.S = S
        # Cycle
        self.C = 2*S - 2
        self.CS = 1 + (S-2)
        self.downward_boundary = S - 1
    
    def _sequence_normalizer(self, x:int) -> list:
        """
        Normalize the position and calculate the starting column for a given index in the zigzag cycle.

        Parameters:
            x (int): The index to normalize.

        Returns:
            list: A list containing the normalized position and the initial column offset [x_n, b].
        """
        # parameters
        c = x // self.C
        r = x % self.C

        # boundaries cases
        if r == 0:
            x_normalized = self.C - 1
            b_normalized = self.CS * (c - 1)
        else:
            x_normalized = r - 1
            b_normalized = self.CS * c
        
        return [x_normalized, b_normalized]

    def _stateless_positioner(self, x:int, b:int) -> list:
        """
        Calculate the 2D position for an element in the zigzag pattern based on the normalized index.

        Parameters:
            x (int): The normalized position index.
            b (int): The starting column based on the normalized cycle.

        Returns:
            list: A list containing the row and column indices [row_index, col_index].

        Raises:
            ValueError: If x is outside the allowed cycle range.
        """
        # check cycle range
        if x < 0 or x >= self.C:
            raise ValueError(f'x is out of the allowed cycle range, which is from 0 to {self.C-1}')
        
        # check boundaries
        if x <= self.downward_boundary:
            return [x, b]

        x_normalized = x - self.downward_boundary
        x_inverted = (self.S - 1) - x_normalized
        return [x_inverted, b + x_normalized]

    def positioner(self, x:int) -> list:
        """
        It coordinates the normalization from _sequence_normalizer, with the stateless positioner from _stateless_positioner
        """
        x_normalized, b_normalized = self._sequence_normalizer(x)
        return self._stateless_positioner(x_normalized, b_normalized)

<>:2: SyntaxWarning: invalid escape sequence '\_'
<>:2: SyntaxWarning: invalid escape sequence '\_'
C:\Users\Lucas Masiero\AppData\Local\Temp\ipykernel_19520\2171179165.py:2: SyntaxWarning: invalid escape sequence '\_'
  """


In [29]:
'''
Test the ZigZagPositioner class

Table Cases
• S = 4

x (n° element, not position)  |   r, c
------------------------------|----------
1                             |   (0, 0)
2                             |   (1, 0)
3                             |   (2, 0)
4                             |   (3, 0)
5                             |   (2, 1)
6                             |   (1, 2)
7                             |   (0, 3)
8                             |   (1, 3)
9                             |   (2, 3)
10                            |   (3, 3)
11                            |   (2, 4)
12                            |   (1, 5)

x element is normalized also into positional terms for the stateless positioner that works with positions rather than n° of the element
'''
zig_zag_positioner = ZigZagPositioner(4)
coordinates = []

for i in range(1, 13):
    coordinates.append(zig_zag_positioner.positioner(i))

coordinates

[[0, 0],
 [1, 0],
 [2, 0],
 [3, 0],
 [2, 1],
 [1, 2],
 [0, 3],
 [1, 3],
 [2, 3],
 [3, 3],
 [2, 4],
 [1, 5]]

In [30]:
import numpy as np

array = np.zeros((S, S*2), dtype=int)
array

array([[0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0]])

In [31]:
array_coordinates = np.array(coordinates)
array[array_coordinates[:, 0], array_coordinates[:, 1]] = 1
array

array([[1, 0, 0, 1, 0, 0, 0, 0],
       [1, 0, 1, 1, 0, 1, 0, 0],
       [1, 1, 0, 1, 1, 0, 0, 0],
       [1, 0, 0, 1, 0, 0, 0, 0]])